In [ ]:
#r "nuget: Microsoft.Data.Sqlite, 6.0.2"
#r "nuget: Dapper, 2.0.123"
#r "nuget: Humanizer, 2.14.1"

Installed Packages Dapper, 2.0.123 Humanizer, 2.14.1 Microsoft.Data.Sqlite, 6.0.2

In [ ]:
using Microsoft.Data.Sqlite;
using Dapper;
using Humanizer;
using System.IO;
using System.Reflection;
using System.Globalization;

In [ ]:
record Node(int NodeId, int NodeParentId, string Tag){
    public Node():this(0,0,""){}
}

In [ ]:
List<Node> nodes  = new (){ 
    new(){NodeId=1,NodeParentId=0, Tag="root"} ,
    new(){NodeId=2,NodeParentId=1} ,
    new(){NodeId=3,NodeParentId=1} ,
    new(){NodeId=4,NodeParentId=2} ,
    new(){NodeId=5,NodeParentId=2} 
};

In [ ]:
var connection = new SqliteConnection("Data Source=graph.db");


In [ ]:
string TypeToSql<T>(Dictionary<Type,string> typeMapper=null)
{
    Type type = typeof(T);
    StringBuilder sb = new();

    if (typeMapper is null) //sqlite
    {
        typeMapper = new Dictionary<Type,string>();
        typeMapper.Add(typeof(string), "text");
        typeMapper.Add(typeof(int), "int");
    }

    sb.Append("CREATE TABLE " + type.Name.Pluralize() + " (\n");

    foreach(PropertyInfo propertyinfo in type.GetProperties())
    {
        Type propType = propertyinfo.PropertyType;
        string sqlType = typeMapper.ContainsKey(propType) ? typeMapper[propType] : "text";
        sb.AppendLine("\t" + propertyinfo.Name + " " + sqlType + ",");
    }

    sb.Remove(sb.Length - 2, 2);
    
    sb.AppendLine("\n);");
    return sb.ToString();
}

In [ ]:
connection.Execute("DROP TABLE Nodes;");
connection.Execute(TypeToSql<Node>())

1

In [ ]:
foreach (Node n in nodes){
    connection.Execute("INSERT INTO Nodes VALUEs (@NodeId, @NodeParentId, @Tag);", n);
}

In [ ]:
Dictionary<int, List<int>> GetAdjList(IEnumerable<Node> nodes){
    Dictionary<int, List<int>> output = new();
    foreach(Node node in nodes){
        if (output.ContainsKey(node.NodeParentId)){
            output[node.NodeParentId].Add(node.NodeId);
        }
        else
        {
            output.Add(node.NodeParentId, new List<int>());
            output[node.NodeParentId].Add(node.NodeId);
        }
    }
    return output;
}

string ToMermaid(IEnumerable<Node> nodes){
    StringBuilder sb = new();
    sb.AppendLine("graph LR");
    foreach(Node node in nodes){
        var tag =  node.Tag == ""?node.NodeId.ToString(): node.Tag; 
        sb.AppendLine($"\t{node.NodeParentId} --- {node.NodeId}({tag})");
    }
    return sb.ToString();
}



In [ ]:
class Node<T> {
    public T Value {get; set;}
    public List<Node<T>> Children {get;} 
    public Node () {Children = new List<Node<T>>();}
}

In [ ]:
new Node<string>()